# Data Engineer Certification Sample Practical Exam

HappyPaws, creates fun and educational apps for pet owners. 

HappyPaws wants to help pet owners understand their pets better by tracking their activities and health through the app. 

The data engineering team is responsible for making sure all the pet data from thousands of users is organized and safe, so pet owners can get tips to keep their pets happy and healthy.



## Task

HappyPaws has collected three datasets over the past year: 
 - "pet_activities.csv" which logs daily activities of pets, 
 - "pet_health.csv" which records vet visits and health issues, and 
 - "users.csv" which contains information about the pet owners. 

Each dataset contains unique identifiers for pets and/or their owners. 

The engineers developing the app currently write code to cross reference all of these data sources. 

They want to make things easier by having a single table with all data included.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset. 

The final dataset should provide a comprehensive view of each pet's activities, health records, and owner information. 

- To test your code, your manager will run only the code `all_pet_data('pet_activities.csv', 'pet_health.csv', 'users.csv')`
- Your `all_pet_data()` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.

## Data

The data that has been provided has the following structure

![image](image.png)

The function that you write must return data as described below. There should be a unique row for each activity/health visit. 

Where missing values are permitted, they should be in the default Python format.

|Column Name | Description |
|-------|-------|
| pet_id | Unique identifier for each pet. There should not be any missing values.|
| date | The date of the activity recorded or the date of the health visit, in date format. There should not be any missing values.|
| activity_type | The type of activity, one of 'Walking', 'Playing', 'Resting' or for rows that relate to a health visit, the value 'Health'. Missing values are permitted.|
| duration_minutes | The duration of the activity in minutes. For rows that relate to health visits, this should be 0. Missing values for other activities are permitted.|
| issue | The health issue identified or check-up note. For rows that relate to activities, this should be a missing value. Missing values for health activities are permitted.|
| resolution | The outcome or advice given for the issue. For rows that relate to activities, this should be a missing value. Missing values for health activities are permitted.|
| owner_id | Unique identifier for the pet owner. All pets must have an owner.|
| owner_age_group | The age group of the owner (e.g., 18-25, 26-35, etc.). Missing values are permitted.|
| pet_type | The type of pet (e.g., Dog, Cat). Missing values are permitted.|

# Função automatizada para consolidação dos dados

In [4]:
import pandas as pd
import numpy as np

In [5]:
#Criando a função
def all_pet_data(df_activities, df_health, df_users):
    
    #Lendo os dados
    pet_activities = pd.read_csv(df_activities)
    pet_health = pd.read_csv(df_health)
    users = pd.read_csv(df_users)
    
    #Modificando as colunas - Pet Activities
    pet_activities = pet_activities.dropna(subset=["pet_id","date"])
    pet_activities['activity_type'] =   pet_activities['activity_type'].str.replace(r'\bWalk\b','Walking', regex=True)
    pet_activities['activity_type'] = pet_activities['activity_type'].str.replace(r'\bRest\b','Resting', regex=True)
    pet_activities['activity_type'] = pet_activities['activity_type'].str.replace(r'\bPlay\b','Playing', regex=True)
    pet_activities['duration_minutes'].replace('-',np.nan,inplace=True)
    pet_activities['duration_minutes'] = pd.to_numeric(pet_activities['duration_minutes'], errors='coerce', downcast='integer')
    pet_activities['date'] = pd.to_datetime(pet_activities['date'])

    #Criando novas colunas - Pet Activities
    pet_activities['issue'] = None
    pet_activities['resolution'] = None
    
    #Modificando as colunas - Pet Health
    pet_health = pet_health.dropna(subset=["pet_id","visit_date"])
    pet_health['date'] = pd.to_datetime(pet_health['visit_date'])

    #Criando novas colunas - Pet Health
    pet_health['activity_type'] = 'Health'
    pet_health['duration_minutes'] = 0

    #Padronizando o df - Pet Health
    pet_health= pet_health[['pet_id','date','activity_type','duration_minutes','issue','resolution']]
    
    #Juntando as bases - Pet Activities + Pet Health
    data = pd.concat([pet_activities,pet_health])
    data = pd.merge(data, users, how='left',on='pet_id')
    
    #Juntando as bases - Join com Users
    data = pd.concat([pet_activities,pet_health])
    data = pd.merge(data, users, how='left',on='pet_id')
    
    return data

In [6]:
all_pet_data('pet_activities.csv', 'pet_health.csv', 'users.csv')

,pet_id,date,activity_type,duration_minutes,issue,resolution,owner_id,owner_age_group,pet_type
0,9930,2023-08-01,Resting,NaN,None,None,7016,18-25,Dog
1,3040,2023-07-24,Resting,4.0,None,None,5637,18-25,Rabbit
2,2381,2023-01-11,Walking,24.0,None,None,5366,18-25,Rabbit
3,3936,2022-09-15,Walking,73.0,None,None,7063,26-35,Hamster
4,8989,2023-02-06,Playing,77.0,None,None,3803,36-45,Dog
...,...,...,...,...,...,...,...,...,...
1873,9136,2022-04-15,Health,0.0,Annual Checkup,"All clear, healthy diet recommended",3553,18-25,Dog
1874,3228,2023-07-30,Health,0.0,Injury,Physical therapy recommended for joint issues,8432,66+,Fish
1875,9163,2023-01-20,Health,0.0,Annual Checkup,"Vaccination completed, next due in 1 year",8254,46-55,Hamster
1876,2805,2022-05-28,Health,0.0,Injury,"Sprain identified, rest and limited movement a...",6327,36-45,Rabbit
